# Case Final
## Análise de Transações PIX
CRISP-DM - https://www.escoladnc.com.br/blog/data-science/metodologia-crisp-dm/
### Objetivos
- Limpar e pré-processar os dados das transações PIX
- Analisar padrões de uso do PIX, tais como os canais mais utilizados e os valores de transação mais comuns
- Use o PySpark MLlib para treinar e avaliar um modelo de detecção de fraude
- Avaliar o desempenho do modelo e fazer recomendações para melhorias futuras

### Dados

O conjunto de dados inclui as seguintes informações para cada transação:
- Detalhes da transação: valor, tempo, remetente e receptor CPF/CNPJ, tipo
- Etiqueta de fraude: uma variável binária que indica se a transação foi fraudulenta (1) ou não (0)

### Tarefas
- Normalização dos dados:
  - O dataset que você lerá está em formato json.
  ```json
  {
      "id_transacao": inteiro,
      "valor": texto,
      "remetente": {
          "nome": texto,
          "banco": texto,
          "tipo": texto
      },
      "destinatario": {
          "nome": texto,
          "banco":texto,
          "tipo": texto
      },          
      "categoria":texto,
      "chave_pix":texto,
      "transaction_date":texto,
      "fraude":inteiro,
  }
    ```
  - Faça sua transformação para formato colunar
- Análise Exploratória de Dados: Use o PySpark para analisar padrões de uso do PIX:
  - chaves pix mais usadas;
  - os valores de transação mais comuns;
  - distribuição dos valores de transação por hora e dia;
  - quais bancos receberam mais transferências por dia;
  - para qual tipo de pessoa (PF ou PJ) foram realizadas mais transações
- Engenharia de Recursos: Apresentar novas características que podem ser úteis para a detecção de fraudes, tais como o número de transações feitas pelo mesmo remetente em um período de tempo específico.
- Modelagem: Use o PySpark MLlib para treinar e detectar possíveis transações que contenham fraude.

### Observação
É importante notar que este é um caso simplificado, e em cenários do mundo real você teria que lidar com dados mais complexos, usar técnicas mais avançadas como métodos de conjuntos e considerar o conhecimento de domínio, bem como leis e regulamentos das instituições financeiras no Brasil.


### Observação II
Não existe resposta 100% correta. É necessário que você use seu pensamento crítico para definir as melhores métricas e análises para o caso.

# Entendimento do Negócio
Você trabalha em um banco e o principal meio de pagamento utilizado no seu banco é o Pix.

Através da base de transações do pix o banco deseja entender qual é o perfil dos clientes que utilizam o pix, além de verificar possíveis transações que tenham fraude. Porém, eles tem um cliente específico que tem um relacionamento muito bom para o banco, por isso, você recebeu a base de transações de cliente dos últimos 2 anos e precisa a partir dela criar um relatório contendo as principais características das transações.


Então, resumindo, temos dois principais objetivos para esse case:
1. Obter valor a partir dos dados
  - Para qual banco esse cliente mais transfere?
  - Qual é a média de transferências por período que esse cliente faz?
  - Baseando-se no valor das transferências, poderia dar um aumento de crédito?
  - Para o que esse cliente mais usa as transferências?
2. Executar um algoritmo de machine learning que identifique possíveis transações com fraude.
3. Pós Processamento
  - Defina ao mínimo cinco métricas de qualidade para seus dados
  - Explique se os seus dados estão com uma boa qualidade

# Preparação do Ambiente de Desenvolvimento

In [1]:
# !pip install pyspark
# !pip install pandas


In [2]:
# Importar bibliotecas e preparar todo o processo de seleção dos dados com spark
# Importar bibliotecas

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import 
# from google.colab import files

# upload = files.upload()



In [3]:
# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2Wlm9bDY0eka79Pd7tED4KnBUZv_24dzLdjRdSZXTLHaKUxpS
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Authtoken saved to configuration file: /home/valdir/.ngrok2/ngrok.yml
https://cdb8-2804-88c-4d3b-d200-2391-96b5-7aa-8632.ngrok-free.app


In [4]:
#iniciar sessão spark
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .config('spark.ui.port', '4050')
    .appName('Case_Final').getOrCreate()
)

23/11/02 12:53:39 WARN Utils: Your hostname, valdir-G5-5590 resolves to a loopback address: 127.0.1.1; using 192.168.0.43 instead (on interface wlp4s0)
23/11/02 12:53:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 12:53:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# carregar os dados
schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType()),
])

schema_base_pix = StructType([
    StructField("id_transacao", IntegerType()),
    StructField("valor", DoubleType()),
    StructField("data", StringType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

df = spark.read.json(
    'case_final.json',
    schema=schema_base_pix,
    timestampFormat='yyyy-MM-dd HH:mm:ss',

)

In [6]:
df.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- data: string (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)



In [7]:
df.count()


100000

# Data Undesrtanting

Primeiramente, devemos entender tudo sobre a fonte dos dados
- Como o dado chega até nós?
- Qual formato virá?
- Aonde o processamento será executado (AWS EMR, Cluster On-Premise)?
- De quanto em quanto tempo eu preciso gerar esse relatório (mensal, diário, near-real time)?

```json
{
  "id_transacao": inteiro,
  "valor": texto,
  "remetente": {
      "nome": texto,
      "banco": texto,
      "tipo": texto
  },
  "destinatario": {
      "nome": texto,
      "banco":texto,
      "tipo": texto
  },
  "categoria": texto,
  "transaction_date":texto,
  "chave_pix":texto,
  "fraude":inteiro,
}
```



# Preparação dos Dados
Agora é hora de começar a preparar os dados de acordo com as suas necessidades.

# Modelagem
Aqui você encontrará utilidade para os dados levantados.

Aqui será onde teremos insights e, a partir desses, novos conhecimentos sobre o business (se tudo até aqui foi feito corretamente).


- Para qual banco esse cliente mais transfere?
- Qual é a média de transferências por período que esse cliente faz?
- Baseando-se no valor das transferências, poderia dar um aumento de crédito?
- Para o que esse cliente mais usa as transferências?
- Executar um algoritmo de machine learning que identifique possíveis transações com fraude.


In [8]:
#transformar remetenete e destinatario em colunas
df = df.withColumn('nome_remetente', df.remetente.nome)
df = df.withColumn('banco_remetente', df.remetente.banco)
df = df.withColumn('tipo_remetente', df.remetente.tipo)
df = df.withColumn('nome_destinatario', df.destinatario.nome)
df = df.withColumn('banco_destinatario', df.destinatario.banco)
df = df.withColumn('tipo_destinatario', df.destinatario.tipo).drop('remetente', 'destinatario')


In [9]:
# Verificar se existem valores nulos
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------------+-----+------+---------+---------+----------------+------+--------------+---------------+--------------+-----------------+------------------+-----------------+
|id_transacao|valor|  data|chave_pix|categoria|transaction_date|fraude|nome_remetente|banco_remetente|tipo_remetente|nome_destinatario|banco_destinatario|tipo_destinatario|
+------------+-----+------+---------+---------+----------------+------+--------------+---------------+--------------+-----------------+------------------+-----------------+
|           0|    0|100000|        0|        0|               0|     0|             0|              0|             0|                0|                 0|                0|
+------------+-----+------+---------+---------+----------------+------+--------------+---------------+--------------+-----------------+------------------+-----------------+



In [10]:
# Para qual banco esse cliente mais transfere?

df.groupBy('banco_destinatario').count().orderBy('count', ascending=False).show()

+------------------+-----+
|banco_destinatario|count|
+------------------+-----+
|                XP|14401|
|               BTG|14390|
|            Nubank|14297|
|              Itau|14281|
|             Caixa|14240|
|                C6|14204|
|          Bradesco|14187|
+------------------+-----+



In [11]:
# Qual é a média de transferências por período que esse cliente faz?
df.groupBy(date_format(col('transaction_date'), 'yyyy-MM').alias('mes')).count().orderBy('mes').show()

+-------+-----+
|    mes|count|
+-------+-----+
|2021-01| 2415|
|2021-02| 3794|
|2021-03| 4179|
|2021-04| 4061|
|2021-05| 4303|
|2021-06| 4238|
|2021-07| 4159|
|2021-08| 4250|
|2021-09| 4096|
|2021-10| 4234|
|2021-11| 4050|
|2021-12| 4232|
|2022-01| 4239|
|2022-02| 3855|
|2022-03| 4283|
|2022-04| 4108|
|2022-05| 4241|
|2022-06| 4091|
|2022-07| 4336|
|2022-08| 4228|
+-------+-----+
only showing top 20 rows



In [12]:
df.groupBy(date_format(col('transaction_date'), 'yyyy').alias('ano')).count().orderBy('ano').show()

+----+-----+
| ano|count|
+----+-----+
|2021|48011|
|2022|50030|
|2023| 1959|
+----+-----+



In [13]:

df.groupBy(date_format(col('transaction_date'),'yyyy').alias('ano')).agg(avg('valor')).show()

+----+------------------+
| ano|        avg(valor)|
+----+------------------+
|2022|10265.353683190187|
|2023|10002.365324144983|
|2021|10355.243481285566|
+----+------------------+



In [14]:
# Para o que esse cliente mais usa as transferências?
df.groupBy('categoria').agg(avg('valor').alias('media')).orderBy('media', ascending=False).show()



+-------------+------------------+
|    categoria|             media|
+-------------+------------------+
|transferencia|34877.567212657654|
|    presentes|2287.6823222390217|
|    vestuario|2269.6187635483498|
|   transporte|2232.2969587966013|
|       outros| 2218.729851764958|
|  alimentacao| 2217.824653330547|
|     educacao|2209.8863816067596|
|        saude|2198.5639014351987|
|        lazer|2154.2605853761643|
+-------------+------------------+



In [15]:
# # Baseando-se no valor das transferências, poderia dar um aumento de crédito?
df.groupBy('banco_destinatario').sum('valor').orderBy('sum(valor)', ascending=False).show()

+------------------+--------------------+
|banco_destinatario|          sum(valor)|
+------------------+--------------------+
|          Bradesco|1.4987422862999955E8|
|                XP|1.4873455870999998E8|
|            Nubank| 1.474946488100003E8|
|                C6| 1.464361347999994E8|
|              Itau|1.4610714452000046E8|
|             Caixa| 1.460292635799994E8|
|               BTG| 1.456598941699996E8|
+------------------+--------------------+



In [16]:
df.describe().show()

23/11/02 12:54:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+----+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|summary|     id_transacao|             valor|data|chave_pix|  categoria|   transaction_date|            fraude|    nome_remetente|banco_remetente|tipo_remetente|nome_destinatario|banco_destinatario|tipo_destinatario|
+-------+-----------------+------------------+----+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|  count|           100000|            100000|   0|   100000|     100000|             100000|            100000|            100000|         100000|        100000|           100000|            100000|           100000|
|   mean|          50999.5|10303.358732200059|NULL|     NULL|       NULL|               NULL|           0.15367|              NU

In [17]:
df.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- data: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- nome_remetente: string (nullable = true)
 |-- banco_remetente: string (nullable = true)
 |-- tipo_remetente: string (nullable = true)
 |-- nome_destinatario: string (nullable = true)
 |-- banco_destinatario: string (nullable = true)
 |-- tipo_destinatario: string (nullable = true)



In [18]:
dfml =  df.drop('remetente', 'id_transacao')

In [19]:
# Executar um algoritmo de machine learning que identifique possíveis transações com fraude.  (Opcional)


from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

# 1.1 - Cria o objeto StringIndexer
indexer = StringIndexer(
    inputCols=[
        "categoria",
        "banco_destinatario",
        "nome_destinatario",
        "tipo_destinatario",
        "chave_pix"
    ],
    outputCols=[
        "categoria_index",
        "banco_destinatario_index",
        "nome_destinatario_index",
        "tipo_destinatario_index",
        "chave_pix_index"
    ]

    )
# Treina o StringIndexer
si = indexer.fit(df)




In [20]:
df_indexer = si.transform(dfml)

In [21]:
df_indexer.show()

23/11/02 12:54:25 WARN DAGScheduler: Broadcasting large task binary with size 1278.6 KiB


+------------------+----+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+---------------+------------------------+-----------------------+-----------------------+---------------+
|             valor|data|chave_pix|    categoria|   transaction_date|fraude|    nome_remetente|banco_remetente|tipo_remetente|   nome_destinatario|banco_destinatario|tipo_destinatario|categoria_index|banco_destinatario_index|nome_destinatario_index|tipo_destinatario_index|chave_pix_index|
+------------------+----+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+---------------+------------------------+-----------------------+-----------------------+---------------+
|            588.08|NULL|aleatoria|       outros|2021-07-16 05:00:55|     0|Jonathan Gonsalves|            BTG|            PF|    

In [22]:
cols_para_filtrar = [
    'valor',
    'transaction_date',
    "categoria_index",
    "banco_destinatario_index",
    "nome_destinatario_index",
    "tipo_destinatario_index",
    "chave_pix_index",
    'fraude'
]

In [23]:
is_fraude = df_indexer.select(cols_para_filtrar).filter('fraude == 1')
no_fraude = df_indexer.select(cols_para_filtrar).filter('fraude == 0')
no_fraude =  no_fraude.sample(False, 0.01, seed=123)

In [24]:
df_concat = no_fraude.union(is_fraude)
dfml = df_concat.sort("transaction_date")
dfml.count()

16202

In [25]:
treino, teste =  dfml.randomSplit([0.70,0.30], seed = 123)
print(f"Train {treino.count()}", f"Test {teste.count()}")

23/11/02 12:54:30 WARN DAGScheduler: Broadcasting large task binary with size 1289.9 KiB
23/11/02 12:54:33 WARN DAGScheduler: Broadcasting large task binary with size 1290.5 KiB
23/11/02 12:54:34 WARN DAGScheduler: Broadcasting large task binary with size 1302.2 KiB
23/11/02 12:54:37 WARN DAGScheduler: Broadcasting large task binary with size 1296.9 KiB
23/11/02 12:54:38 WARN DAGScheduler: Broadcasting large task binary with size 1289.9 KiB
23/11/02 12:54:41 WARN DAGScheduler: Broadcasting large task binary with size 1290.5 KiB
23/11/02 12:54:42 WARN DAGScheduler: Broadcasting large task binary with size 1302.2 KiB
23/11/02 12:54:45 WARN DAGScheduler: Broadcasting large task binary with size 1296.9 KiB


Train 11278 Test 4924


In [26]:
is_fraude = udf(lambda fraude: 1.0 if fraude >0 else 0.0, DoubleType())
treino = treino.withColumn("is_fraude", is_fraude(treino.fraude))

In [27]:
# criar os vetores de recursos.
# VectorAssembler é um transformador que combina uma determinada lista de colunas em uma única coluna de vetor.
assembler = VectorAssembler(
  inputCols = [x for x in treino.columns if x not in ["transaction_date", "fraude", "is_fraude"]],
  outputCol = "features")

# Use Logistic Regression.
# is a machine learning algorithm that is used for classification tasks
lr = LogisticRegression().setParams(
    maxIter = 100000,
    labelCol = "is_fraude",
    predictionCol = "prediction")


# This will train a logistic regression model on the input data and return a 
# LogisticRegressionModel object which can be used to make predictions on new data.
model = Pipeline(stages = [assembler, lr]).fit(treino)

23/11/02 12:54:47 WARN DAGScheduler: Broadcasting large task binary with size 1289.9 KiB
23/11/02 12:54:49 WARN DAGScheduler: Broadcasting large task binary with size 1290.5 KiB
23/11/02 12:54:50 WARN DAGScheduler: Broadcasting large task binary with size 1302.2 KiB
23/11/02 12:54:53 WARN DAGScheduler: Broadcasting large task binary with size 1289.9 KiB
23/11/02 12:54:55 WARN DAGScheduler: Broadcasting large task binary with size 1290.5 KiB
23/11/02 12:54:56 WARN DAGScheduler: Broadcasting large task binary with size 1302.2 KiB
23/11/02 12:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1340.9 KiB
23/11/02 12:55:06 WARN DAGScheduler: Broadcasting large task binary with size 1341.6 KiB
23/11/02 12:55:08 WARN DAGScheduler: Broadcasting large task binary with size 1341.6 KiB
23/11/02 12:55:08 WARN DAGScheduler: Broadcasting large task binary with size 1341.6 KiB
23/11/02 12:55:08 WARN DAGScheduler: Broadcasting large task binary with size 1341.6 KiB
23/11/02 12:55:09 WAR

In [30]:
predicted = model.transform(teste)
predicted = predicted.withColumn("is_fraude", is_fraude(predicted.fraude))
predicted.crosstab("is_fraude","prediction").show()

23/11/02 12:56:05 WARN DAGScheduler: Broadcasting large task binary with size 1289.9 KiB
23/11/02 12:56:07 WARN DAGScheduler: Broadcasting large task binary with size 1290.5 KiB
23/11/02 12:56:08 WARN DAGScheduler: Broadcasting large task binary with size 1302.2 KiB
23/11/02 12:56:10 WARN DAGScheduler: Broadcasting large task binary with size 1339.9 KiB
23/11/02 12:56:10 WARN DAGScheduler: Broadcasting large task binary with size 1330.6 KiB
23/11/02 12:56:10 WARN DAGScheduler: Broadcasting large task binary with size 1330.3 KiB
23/11/02 12:56:10 WARN DAGScheduler: Broadcasting large task binary with size 1328.0 KiB
23/11/02 12:56:11 WARN DAGScheduler: Broadcasting large task binary with size 1289.9 KiB
23/11/02 12:56:12 WARN DAGScheduler: Broadcasting large task binary with size 1290.5 KiB
23/11/02 12:56:13 WARN DAGScheduler: Broadcasting large task binary with size 1302.2 KiB
23/11/02 12:56:16 WARN DAGScheduler: Broadcasting large task binary with size 1346.7 KiB


+--------------------+---+----+
|is_fraude_prediction|0.0| 1.0|
+--------------------+---+----+
|                 1.0|  0|4660|
|                 0.0|262|   2|
+--------------------+---+----+



23/11/02 12:56:16 WARN DAGScheduler: Broadcasting large task binary with size 1334.5 KiB
23/11/02 12:56:16 WARN DAGScheduler: Broadcasting large task binary with size 1337.8 KiB


# Avaliação do Modelo
Será que seu modelo atinge todas as necessidades que foram definidas inicialmente? (e.g. pessoa em cima da bicicleta muda o resultado final)



# Deployment
Apresente o relatório com os resultados obtidos.




  